In [1]:
import numpy as np                     # Numerical algorithms on arrays
import cv2                             # OpenCV
from matplotlib import pyplot as plt   # Plot library
import matplotlib.cm as cm             # Image color map 
import tools                           # A few helpers to plot multiple images
from PIL import Image                  # For rescaling images

In [2]:
# Load image in grayscale format
road = cv2.imread('Images/Road.png', cv2.IMREAD_GRAYSCALE)
building = cv2.imread('Images/Building.png', cv2.IMREAD_GRAYSCALE)

road_color = cv2.imread('Images/Road.png', cv2.IMREAD_COLOR)
building = cv2.imread('Images/Building.png', cv2.IMREAD_COLOR)

In [ ]:
# Show images
cv2.imshow('Color road displayed in grayscale', road)
cv2.imshow('Color building displayed in grayscale', building)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [7]:
# rescaling road

basewidth = 300
road_img = Image.open('Images/Road.png')
wpercent = (basewidth/float(road_img.size[0]))
hsize = int((float(road_img.size[1])*float(wpercent)))
resc_road = road_img.resize((basewidth,hsize), Image.ANTIALIAS)
resc_road.save('rescale_road.png', quality = 95)
